In [1]:
import habanero
from habanero import Crossref
from pprint import pprint
from collections import Counter # for easy counting
import ast # for string to dictionary conversion
import pandas as pd # for data manipulation
import numpy as np # for array manipulation

import urllib.request
from urllib.error import HTTPError
import sys

import arxiv

Inspiration: https://colab.research.google.com/github/EPS-Libraries-Berkeley/volt/blob/main/Search/crossref_library_module.ipynb#scrollTo=scyWPCPqcdP8

## Get info from DOI

In [3]:
cr = Crossref() # create a crossref object

In [4]:
# query for the term "permafrost"
xai = cr.works(query = "xai")

In [106]:
dois= ['10.1016/j.artint.2021.103457','10.1109/MIS.2019.2957223','10.3390/e23081064']

In [385]:
dois=[
"10.1007/978-3-030-10997-4_13",
"10.1145/3236009",
"https://ercim-news.ercim.eu/images/stories/EN116/EN116-web.pdf#page=12",
"10.1007/978-3-030-16148-4_5",
"10.1007/978-3-030-24409-5_9",
"http://www.dpceonline.it/index.php/dpceonline/article/view/1083",
"10.1007/978-3-030-61527-7_24",
"http://ceur-ws.org/Vol-2646/18-paper.pdf",
"10.1016/j.artint.2020.103428",
"10.1109/ICDM50108.2020.00122",
"TBD - Submitted",
"10.1609/aaai.v33i01.33019780",
"10.1007/978-3-030-46150-8_12",
"10.1609/aaai.v34i09.7116",
"http://hdl.handle.net/11568/1088440",
"10.1145/3351095.3372855",
"10.1007/978-3-030-43823-4_14",
"10.1007/978-3-030-61527-7_27",
"10.1007/978-3-030-65965-3_34",
"10.1109/CogMI50398.2020.00029",
"10.1007/978-3-030-69128-8_2",
"no doi reference",
"no doi reference",
"10.1016/j.ipm.2021.102657",
"https://www.ildiabeteonline.it/intelligenza-artificiale-in-ambito-diabetologico-prospettive-dalla-ricerca-di-base-alle-applicazioni-cliniche/",
"10.1016/j.artint.2021.103457",
"10.1109/MIS.2019.2957223",
"10.3390/e23081064",
"10.1007/978-3-030-88942-5_27",
"arXiv:1805.10820",
"arXiv:2111.11863v1",
"arXiv:1806.09936",
"http://pages.di.unipi.it/ruggieri/Papers/ds2021.pdf",
"10.1109/ISCC53001.2021.9631485",
"10.1145/3461702.3462553",
"10.3389/frai.2021.699448"
]

In [8]:
doi = '10.1140/epjds/s13688-022-00315-z'

paper=cr.works(ids = doi)
papero={}

if 'abstract' in paper['message']:
    papero['abstract']=paper['message']['abstract'].lstrip('<jats:p>').rstrip('</jats:p>')
papero['url']=paper['message']['URL']
papero['title']=paper['message']['title'][0]
papero['year']=paper['message']['indexed']['date-parts'][0][0] #non sono sicuro sia il modo corretto di prendere l'anno
papero['paperType']=paper['message']['type']
papero['container']=paper['message']['container-title'][0]
papero['publisher']=paper['message']['publisher']
#papero['ISSN'] = paper['message']['ISSN'][0]
papero['authors']=''
authorList=paper['message']['author']
for i,author in enumerate(authorList):
    if i== len(authorList)-1:
        papero['authors'] += author['family'] + ' ' + author['given']
    else:
        papero['authors'] += author['family'] + ' ' + author['given'] + ', '
        

print(f"{papero['authors']} ({papero['year']}). {papero['title']} - {papero['paperType']}: {papero['container']}. ")
pprint(papero)
# print(year)

Voukelatou Vasiliki, Miliou Ioanna, Giannotti Fosca, Pappalardo Luca (2022). Understanding peace through the world news - journal-article: EPJ Data Science. 
{'abstract': 'itle>Abstract</jats:title><jats:p>Peace is a principal dimension '
             'of well-being and is the way out of inequity and violence. Thus, '
             'its measurement has drawn the attention of researchers, '
             'policymakers, and peacekeepers. During the last years, novel '
             'digital data streams have drastically changed the research in '
             'this field. The current study exploits information extracted '
             'from a new digital database called Global Data on Events, '
             'Location, and Tone (GDELT) to capture peace through the Global '
             'Peace Index (GPI). Applying predictive machine learning models, '
             'we demonstrate that news media attention from GDELT can be used '
             'as a proxy for measuring GPI at a monthly level. 

In [7]:
papero['abstract']

'itle>Abstract</jats:title><jats:p>Peace is a principal dimension of well-being and is the way out of inequity and violence. Thus, its measurement has drawn the attention of researchers, policymakers, and peacekeepers. During the last years, novel digital data streams have drastically changed the research in this field. The current study exploits information extracted from a new digital database called Global Data on Events, Location, and Tone (GDELT) to capture peace through the Global Peace Index (GPI). Applying predictive machine learning models, we demonstrate that news media attention from GDELT can be used as a proxy for measuring GPI at a monthly level. Additionally, we use explainable AI techniques to obtain the most important variables that drive the predictions. This analysis highlights each country’s profile and provides explanations for the predictions, and particularly for the errors and the events that drive these errors. We believe that digital data exploited by research

In [392]:
papersExcel=[]
papersReference=[]

for i,doi in enumerate(dois):
    try:
        if doi[:6] =="arXiv:":
            papero={}
            doi=doi.lstrip("arXiv:")
            search = arxiv.Search(id_list=[doi])

            #papero = next(search.results())
            paper=list(search.results())[0]
            papero['title']=paper.title

            papero['abstract']=paper.summary
            papero['DOI']=paper.doi
            papero['doi']=doi
            papero['year']=paper.published.year

            authors=paper.authors
            auts=[]
            for el in authors:
                auts.append(el.name)
            papero['authors']=''
            for i,author in enumerate(auts):
                if i== len(auts)-1:
                    papero['authors'] += author
                else:
                    papero['authors'] += author  + ', '       

            papero['container']='Arxive'
            papero['paperType']='Arxive pre-print'


            reference=f"{papero['authors']} ({papero['year']}). {papero['title']} - {papero['container']}. "
            #print(reference)
            papersExcel.append(papero)
            papersReference.append(reference)
        ### DOI Section ###
        paper=cr.works(ids = doi)
        papero={}
        
        papero['title']=paper['message']['title'][0]
        if 'abstract' in paper['message']:
            papero['abstract']=paper['message']['abstract'].lstrip('<jats:p>').rstrip('</jats:p>')
        papero['DOI']=paper['message']['URL']
        papero['doi']=doi
        papero['year']=paper['message']['indexed']['date-parts'][0][0] #non sono sicuro sia il modo corretto di prendere l'anno
        papero['paperType']=paper['message']['type']
        papero['container']=paper['message']['container-title'][0]
        papero['publisher']=paper['message']['publisher']
        papero['ISSN'] = paper['message']['ISSN'][0]
        papero['authors']=''
        authorList=paper['message']['author']
        papero['DOI_exists']='True'
        for i,author in enumerate(authorList):
            if i== len(authorList)-1:
                papero['authors'] += author['family'] + ' ' + author['given']
            else:
                papero['authors'] += author['family'] + ' ' + author['given'] + ', '
        papero['references-count']=paper['message']['references-count']

        reference=f"{papero['authors']} ({papero['year']}). {papero['title']} - {papero['paperType']}: {papero['container']}. "

        papersExcel.append(papero)
        papersReference.append(reference)
    except:
        print(f'DOI {doi} not found.')
        papero={}
        papero['doi']=doi
        papero['DOI_exists']='False'
        papersExcel.append(papero)
        
# pprint(papersExcel)

DOI https://ercim-news.ercim.eu/images/stories/EN116/EN116-web.pdf#page=12 not found.
DOI http://www.dpceonline.it/index.php/dpceonline/article/view/1083 not found.
DOI http://ceur-ws.org/Vol-2646/18-paper.pdf not found.
DOI 10.1109/ICDM50108.2020.00122 not found.
DOI TBD - Submitted not found.
DOI http://hdl.handle.net/11568/1088440 not found.
DOI 10.1145/3351095.3372855 not found.
DOI 10.1109/CogMI50398.2020.00029 not found.
DOI no doi reference not found.
DOI no doi reference not found.
DOI https://www.ildiabeteonline.it/intelligenza-artificiale-in-ambito-diabetologico-prospettive-dalla-ricerca-di-base-alle-applicazioni-cliniche/ not found.
DOI 1805.10820 not found.
DOI 2111.11863v1 not found.
DOI 1806.09936 not found.
DOI http://pages.di.unipi.it/ruggieri/Papers/ds2021.pdf not found.
DOI 10.1109/ISCC53001.2021.9631485 not found.
DOI 10.1145/3461702.3462553 not found.


In [399]:
papersDF=pd.DataFrame(papersExcel)
papersDF.sort_values(by=['paperType', 'year'])
papersDF=papersDF.rename(columns={
    'title':'Title of the scientific publication',
    'DOI':'doi link',
    'doi':'DOI',
    'ISSN':'ISSN or eSSN',
    'authors':'Authors',
    'year':'Number, date',
    'paperType':'Type of scientific publication',
    'publisher':'Publisher',
    'year':'Year of publication',
    'abstract':'abstract'
})
papersDF

,Title of the scientific publication,doi link,DOI,Year of publication,Type of scientific publication,container,Publisher,ISSN or eSSN,Authors,DOI_exists,references-count,abstract
0,Helping Your Docker Images to Spread Based on ...,http://dx.doi.org/10.1007/978-3-030-10997-4_13,10.1007/978-3-030-10997-4_13,2021.0,book-chapter,Machine Learning and Knowledge Discovery in Da...,Springer International Publishing,0302-9743,"Guidotti Riccardo, Soldani Jacopo, Neri Davide...",True,37.0,NaN
1,A Survey of Methods for Explaining Black Box M...,http://dx.doi.org/10.1145/3236009,10.1145/3236009,2022.0,journal-article,ACM Computing Surveys,Association for Computing Machinery (ACM),0360-0300,"Guidotti Riccardo, Monreale Anna, Ruggieri Sal...",True,142.0,"In recent years, many accurate decision suppor..."
2,NaN,NaN,https://ercim-news.ercim.eu/images/stories/EN1...,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN
3,Investigating Neighborhood Generation Methods ...,http://dx.doi.org/10.1007/978-3-030-16148-4_5,10.1007/978-3-030-16148-4_5,2021.0,book-chapter,Advances in Knowledge Discovery and Data Mining,Springer International Publishing,0302-9743,"Guidotti Riccardo, Monreale Anna, Cariaggi Leo...",True,32.0,NaN
4,Explaining Multi-label Black-Box Classifiers f...,http://dx.doi.org/10.1007/978-3-030-24409-5_9,10.1007/978-3-030-24409-5_9,2021.0,book-chapter,Precision Health and Medicine,Springer International Publishing,1860-949X,"Panigutti Cecilia, Guidotti Riccardo, Monreale...",True,26.0,NaN
5,NaN,NaN,http://www.dpceonline.it/index.php/dpceonline/...,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN
6,Explaining Sentiment Classification with Synth...,http://dx.doi.org/10.1007/978-3-030-61527-7_24,10.1007/978-3-030-61527-7_24,2021.0,book-chapter,Discovery Science,Springer International Publishing,0302-9743,"Lampridis Orestis, Guidotti Riccardo, Ruggieri...",True,40.0,itle>Abstract</jats:title>\n<jats:p>We present...
7,NaN,NaN,http://ceur-ws.org/Vol-2646/18-paper.pdf,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN
8,Evaluating local explanation methods on ground...,http://dx.doi.org/10.1016/j.artint.2020.103428,10.1016/j.artint.2020.103428,2021.0,journal-article,Artificial Intelligence,Elsevier BV,0004-3702,Guidotti Riccardo,True,48.0,NaN
9,NaN,NaN,10.1109/ICDM50108.2020.00122,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN


In [398]:
papersDF.to_excel('doi_retrieved_XaiPapers.xlsx')

## Merge old and new paper

In [402]:
sheet_id = '1VaF5jAauYYYKo1WH40kLlsQ0-azIX5eMZoFYv5JrVr0'
sheet_names = ['papers','dataset','algorithms','masterThesis','phdThesis']

In [403]:
sheet_name= sheet_names[0]
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

In [404]:
papersByHand = pd.read_csv(url,header=0)

Index(['id', 'Type of scientific publication',
       'Title of the scientific publication', 'DOI', 'ISSN or eSSN', 'Authors',
       'Title of the journal or equivalent', 'Number, date', 'Publisher',
       'Place of publication', 'Pubblication status', 'Year of publication',
       'Relevant pages', 'Joint Public / Private publication', 'Peer-review',
       'Is/will open access provided to this publication', 'Authors names',
       'Research line', 'List of dataset used in the paper', 'notes',
       'visible on website', 'abstract'],
      dtype='object')

In [469]:
mergedDF=papersDF.merge(papersByHand,on='DOI',indicator=True,how='outer',suffixes=('', '_y'))

In [470]:
for index, row in mergedDF.iterrows():  #copio right su left
    if row['_merge']=='right_only':
        mergedDF.at[index,'Title of the scientific publication']=row['Title of the scientific publication_y']
        mergedDF.at[index,'Year of publication']=row['Year of publication_y']
        mergedDF.at[index,'Type of scientific publication']=row['Type of scientific publication_y']
        mergedDF.at[index,'Publisher']=row['Publisher_y']
        mergedDF.at[index,'ISSN or eSSN']=row['ISSN or eSSN_y']
        mergedDF.at[index,'Publisher']=row['Publisher_y']
        mergedDF.at[index,'Authors']=row['Authors_y']        

In [471]:
mergedDF[mergedDF['_merge']=='right_only']

,Title of the scientific publication,doi link,DOI,Year of publication,Type of scientific publication,container,Publisher,ISSN or eSSN,Authors,DOI_exists,...,Joint Public / Private publication,Peer-review,Is/will open access provided to this publication,Authors names,Research line,List of dataset used in the paper,notes,visible on website,abstract_y,_merge
41,Open the black box data-driven explanation of ...,NaN,arXiv:1806.09936,2018.0,NaN,NaN,NaN,NaN,"Pedreschi, D., Giannotti, F., Guidotti, R., Mo...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NaN,right_only
42,Local rule-based explanations of black box dec...,NaN,arXiv:1805.10820,2018.0,NaN,NaN,NaN,NaN,"Guidotti, R., Monreale, A., Ruggieri, S., Pedr...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NaN,right_only


In [472]:
mergedDF.drop([col for col in mergedDF.columns if '_y' in col], axis=1, inplace=True)
mergedDF

,Title of the scientific publication,doi link,DOI,Year of publication,Type of scientific publication,container,Publisher,ISSN or eSSN,Authors,DOI_exists,...,Relevant pages,Joint Public / Private publication,Peer-review,Is/will open access provided to this publication,Authors names,Research line,List of dataset used in the paper,notes,visible on website,_merge
0,Helping Your Docker Images to Spread Based on ...,http://dx.doi.org/10.1007/978-3-030-10997-4_13,10.1007/978-3-030-10997-4_13,2021.0,book-chapter,Machine Learning and Knowledge Discovery in Da...,Springer International Publishing,0302-9743,"Guidotti Riccardo, Soldani Jacopo, Neri Davide...",True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,both
1,A Survey of Methods for Explaining Black Box M...,http://dx.doi.org/10.1145/3236009,10.1145/3236009,2022.0,journal-article,ACM Computing Surveys,Association for Computing Machinery (ACM),0360-0300,"Guidotti Riccardo, Monreale Anna, Ruggieri Sal...",True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,both
2,NaN,NaN,https://ercim-news.ercim.eu/images/stories/EN1...,NaN,NaN,NaN,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,both
3,Investigating Neighborhood Generation Methods ...,http://dx.doi.org/10.1007/978-3-030-16148-4_5,10.1007/978-3-030-16148-4_5,2021.0,book-chapter,Advances in Knowledge Discovery and Data Mining,Springer International Publishing,0302-9743,"Guidotti Riccardo, Monreale Anna, Cariaggi Leo...",True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,both
4,Explaining Multi-label Black-Box Classifiers f...,http://dx.doi.org/10.1007/978-3-030-24409-5_9,10.1007/978-3-030-24409-5_9,2021.0,book-chapter,Precision Health and Medicine,Springer International Publishing,1860-949X,"Panigutti Cecilia, Guidotti Riccardo, Monreale...",True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,both
5,NaN,NaN,http://www.dpceonline.it/index.php/dpceonline/...,NaN,NaN,NaN,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,both
6,Explaining Sentiment Classification with Synth...,http://dx.doi.org/10.1007/978-3-030-61527-7_24,10.1007/978-3-030-61527-7_24,2021.0,book-chapter,Discovery Science,Springer International Publishing,0302-9743,"Lampridis Orestis, Guidotti Riccardo, Ruggieri...",True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,both
7,NaN,NaN,http://ceur-ws.org/Vol-2646/18-paper.pdf,NaN,NaN,NaN,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,both
8,Evaluating local explanation methods on ground...,http://dx.doi.org/10.1016/j.artint.2020.103428,10.1016/j.artint.2020.103428,2021.0,journal-article,Artificial Intelligence,Elsevier BV,0004-3702,Guidotti Riccardo,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,both
9,NaN,NaN,10.1109/ICDM50108.2020.00122,NaN,NaN,NaN,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,both


In [473]:
mergedDF.to_excel('augmented_XaiPapers.xlsx')

## Doi to bibtex

In [9]:
BASE_URL = 'http://dx.doi.org/'
url = BASE_URL + doi
req = urllib.request.Request(url)
req.add_header('Accept', 'application/x-bibtex')
try:
    with urllib.request.urlopen(req) as f:
        bibtex = f.read().decode()
    print(bibtex)
except HTTPError as e:
    if e.code == 404:
        print('DOI not found.')
    else:
        print('Service unavailable.')
    sys.exit(1)

print(type(bibtex))

@article{Voukelatou_2022,
	doi = {10.1140/epjds/s13688-022-00315-z},
	url = {https://doi.org/10.1140%2Fepjds%2Fs13688-022-00315-z},
	year = 2022,
	month = {jan},
	publisher = {Springer Science and Business Media {LLC}},
	volume = {11},
	number = {1},
	author = {Vasiliki Voukelatou and Ioanna Miliou and Fosca Giannotti and Luca Pappalardo},
	title = {Understanding peace through the world news},
	journal = {{EPJ} Data Science}
}
<class 'str'>


TROVARE IL DOI DI XAI

funder XAI = {'DOI': '10.13039/100010663',
  'award': ['G.A. 834756'],
  'doi-asserted-by': 'publisher',
  'name': 'H2020 European Research Council'}

## Write Google Doc

Create a google DOC like this one
https://docs.google.com/spreadsheets/d/1VaF5jAauYYYKo1WH40kLlsQ0-azIX5eMZoFYv5JrVr0/edit#gid=0

Check update function to write Google DOC

In [92]:
import pygsheets